# Common utitities for all production alphas

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../TrainingAlphas/Alpha.ipynb");

In [ ]:
if !@isdefined source_name
    source_name = "Alpha"    
    @warn "source_name not defined: using $source_name"
end;

In [ ]:
function get_recommendee_username()
    if !@isdefined username
        disk_username = open(get_data_path("recommendations/recommendee_username.txt")) do f
            read(f, String)
        end        
        @warn "username not defined: using $disk_username"
        global username = disk_username
    end
    username
end;

In [ ]:
function recommendee_alpha_basepath()
    username = get_recommendee_username()
    mkpath(get_data_path("recommendations/$username/alphas"))
end;

In [ ]:
redirect_logging(get_data_path("recommendations/$(get_recommendee_username())/alphas/$source_name"));

## Reading and writing data

In [ ]:
function get_recommendee_split(content::String)
    raw_split(content) = get_raw_recommendee_split(content)
    if content == "explicit"
        df = raw_split("explicit")        
    elseif content == "implicit"
        df = cat(
            raw_split("explicit"),
            raw_split("implicit"),
        )
        df.rating .= 1        
    elseif content == "ptw"
        df = raw_split("ptw")                
    else
        @assert false
    end
    df
end

function get_raw_recommendee_split(content::String)
    @assert content in ["explicit", "implicit", "ptw"]
    username = get_recommendee_username()
    file = get_data_path("recommendations/$username/$content.jld2")
    JLD2.load(file, "dataset")
end;

In [ ]:
function write_recommendee_alpha(ratings::Vector{Float32}, outdir::String)
    @assert length(ratings) == num_items()
    alpha = RatingsDataset(rating = ratings)
    outdir = mkpath("$(recommendee_alpha_basepath())/$outdir")
    JLD2.save("$outdir/alpha.jld2", Dict("alpha" => alpha))
end;

In [ ]:
function read_recommendee_alpha(alpha::String, content::String)
    @assert content in ["explicit", "implicit", "ptw", "all"]
    df = JLD2.load("$(recommendee_alpha_basepath())/$alpha/alpha.jld2")["alpha"]
    df = RatingsDataset(user=fill(1, num_items()), item = 1:num_items(), rating = df.rating)
    if content == "all"
        return df
    else
        mask = get_recommendee_split(content).item
        return RatingsDataset(df.user[mask], df.item[mask], df.rating[mask])
    end
end;

In [ ]:
function read_recommendee_alpha(alphas::Vector{String}, content::String, implicit::Bool)
    _, β = regress(alphas, content, implicit)
    ratings = zeros(Float32, num_items())
    for i = 1:length(alphas)
        ratings += β[i] .* read_recommendee_alpha(alphas[i], "all").rating
    end
    if implicit
        @assert length(alphas) + 1 == length(β)
        ratings .+= β[end] * 1 / num_items()
    end

    df = get_recommendee_split(content)
    RatingsDataset(user=df.user, item=df.item, rating=ratings[df.item])
end;

## Dependecy management

In [ ]:
function reset_recommenndee_alphas()
    rm(recommendee_alpha_basepath(), recursive = true)
end;